<a href="https://colab.research.google.com/github/Ihalagedara/optimization_agent/blob/main/anomaly_detector_in_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# -----------------------------
# 1. Load Data
# -----------------------------
# Good KPI parameter sets (rows = cells, cols = params)
df_good = pd.read_csv("good_parameters.csv")

# Degraded parameter set (one row)
df_bad = pd.read_csv("degraded_parameters.csv")

# -----------------------------
# 2. Preprocessing
# -----------------------------
scaler = StandardScaler()
X_good = scaler.fit_transform(df_good.values)
X_bad = scaler.transform(df_bad.values)

n_features = X_good.shape[1]

# -----------------------------
# 3. Build Autoencoder
# -----------------------------
input_layer = Input(shape=(n_features,))
encoded = Dense(64, activation="relu")(input_layer)
encoded = Dense(32, activation="relu")(encoded)
bottleneck = Dense(8, activation="relu")(encoded)  # compressed representation
decoded = Dense(32, activation="relu")(bottleneck)
decoded = Dense(64, activation="relu")(decoded)
output_layer = Dense(n_features, activation="linear")(decoded)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

# -----------------------------
# 4. Train Autoencoder
# -----------------------------
history = autoencoder.fit(
    X_good, X_good,
    epochs=100,
    batch_size=16,
    shuffle=True,
    validation_split=0.1,
    verbose=0
)

# -----------------------------
# 5. Apply to Degraded Case
# -----------------------------
reconstructed = autoencoder.predict(X_bad)
reconstruction_error = np.abs(X_bad - reconstructed)

# -----------------------------
# 6. Identify Abnormal Parameters
# -----------------------------
param_errors = reconstruction_error[0]
param_importance = np.argsort(param_errors)[::-1]

print("\nParameters ranked by abnormality (highest error first):")
for idx in param_importance:
    col = df_good.columns[idx]
    diff = X_bad[0][idx] - reconstructed[0][idx]
    direction = "Increase" if diff < 0 else "Decrease"
    print(f"- {col}: {direction} (Current={df_bad.iloc[0, idx]}, Suggested≈{scaler.inverse_transform(reconstructed)[0][idx]:.2f})")

# -----------------------------
# 7. Generate Suggested Parameter Values
# -----------------------------
# Replace degraded params with reconstructed values (closer to good configs)
suggested_params = scaler.inverse_transform(reconstructed)
suggested_df = pd.DataFrame(suggested_params, columns=df_good.columns)

print("\nSuggested new parameter values:")
print(suggested_df.round(2))
